# Chess-Bot

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

import chessbot as cb

## Environment

In [2]:
cb.environment.create()

Dataset archive file found at /home/wumphlett/personal/Chess-Bot/data/CCRL-4040.[1505357].pgn.7z
Dataset file found at /home/wumphlett/personal/Chess-Bot/data/CCRL-4040.[1505357].pgn
Dataset pkl files found at /home/wumphlett/personal/Chess-Bot/data
Dataset val pkl files found at /home/wumphlett/personal/Chess-Bot/data
Done


## Pos2Vec

In [3]:
train, val = cb.dataset.pos2vec_dataset()

In [4]:
pos2vec = cb.model.train_pos2vec(train, val)

Training layer 1/4
Epoch 1/200
40000/40000 [==============================] - 74s 2ms/step - loss: 0.3156 - val_loss: 0.1197 - lr: 0.0050
Epoch 2/200
40000/40000 [==============================] - 74s 2ms/step - loss: 0.1074 - val_loss: 0.1005 - lr: 0.0049
Epoch 3/200
40000/40000 [==============================] - 74s 2ms/step - loss: 0.0972 - val_loss: 0.0944 - lr: 0.0048
Epoch 4/200
40000/40000 [==============================] - 73s 2ms/step - loss: 0.0921 - val_loss: 0.0900 - lr: 0.0047
Epoch 5/200
40000/40000 [==============================] - 74s 2ms/step - loss: 0.0882 - val_loss: 0.0865 - lr: 0.0046
Epoch 6/200
40000/40000 [==============================] - 72s 2ms/step - loss: 0.0851 - val_loss: 0.0838 - lr: 0.0045
Epoch 7/200
40000/40000 [==============================] - 72s 2ms/step - loss: 0.0827 - val_loss: 0.0817 - lr: 0.0044
Epoch 8/200
40000/40000 [==============================] - 73s 2ms/step - loss: 0.0808 - val_loss: 0.0799 - lr: 0.0043
Epoch 9/200
40000/40000 [====

KeyboardInterrupt: 

In [ ]:
import numpy as np

ex = np.reshape(x_train[0][0][0], (1, 773))

print("ex")
print(ex)

print("enc")
print(pos2vec(ex)[0])

## Sandbox